# Portfolio

In [1]:
import pandas as pd
from beakerx import *

In [2]:
prices=pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [3]:
SimpleTimePlot(prices, prices.keys())

/opt/conda/lib/python3.7/site-packages/beakerx/plot/chart.py:312: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  xs = tableData.index.get_values()


In [4]:
from pyutil.portfolio.portfolio import Portfolio

# construct a 1/n portfolio, assets that come in later will initially not have any weight
def f(x):
    # how many assets are alive?
    n = x.notnull().sum()
    y = pd.Series(index=x.index)
    if n > 0:
        y[x.notnull()] = 1.0/n
    return y
    
p = Portfolio(prices, weights=prices.ffill().apply(f, axis=1))


In [5]:
print(dir(p))

['_Portfolio__before', '_Portfolio__prices', '_Portfolio__series2frame', '_Portfolio__weights', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'apply', 'asset_returns', 'assets', 'cash', 'copy', 'empty', 'forward', 'fromPosition', 'head', 'index', 'iron_threshold', 'iron_time', 'last_dates', 'leverage', 'merge', 'nav', 'position', 'prices', 'read_csv', 'rename', 'sector', 'state', 'subportfolio', 'tail', 'to_csv', 'to_frame', 'trading_days', 'truncate', 'weight_current', 'weighted_returns', 'weights']


## Sector analysis

It is possible to assign each asset to a sector via sectormaps. 


In [10]:
sectormap = {"A": "S1", "B": "S1", "C": "S2", "D": "S2", "E": "S2", "F": "S2", "G": "S3"}
p.tail(5).sector(symbolmap=sectormap)

,S1,S2,S3
2015-04-16,0.285714,0.571429,0.142857
2015-04-17,0.285714,0.571429,0.142857
2015-04-20,0.285714,0.571429,0.142857
2015-04-21,0.285714,0.571429,0.142857
2015-04-22,0.285714,0.571429,0.142857


In [11]:
TableDisplay(p.state)

## Top/Flop

In [12]:
p.top_flop().ytd.top

AttributeError: 'Portfolio' object has no attribute 'top_flop'

## Rebalancing

Daily rebalancing is somewhat expensive. It is possible to "iron" the portfolio and rebalance either
* on a fixed grid in time
* with respect to threshold that shall not be exceeded

In [13]:
x = p.iron_time("3M")

In [14]:
# The pro
x.trading_days

[Timestamp('2013-01-31 00:00:00'),
 Timestamp('2013-04-30 00:00:00'),
 Timestamp('2013-07-31 00:00:00'),
 Timestamp('2013-10-31 00:00:00'),
 Timestamp('2014-01-31 00:00:00'),
 Timestamp('2014-04-30 00:00:00'),
 Timestamp('2014-07-31 00:00:00'),
 Timestamp('2014-10-31 00:00:00'),
 Timestamp('2015-01-30 00:00:00'),
 Timestamp('2015-04-22 00:00:00')]

In [10]:
# The portfolio is rebalanced at 
x.state

,Month-to-Date,Year-to-Date,30-Apr-14,31-Jul-14,31-Oct-14,30-Jan-15,22-Apr-15,Extrapolated,Gap,group,Sector Weight,Relative Sector
Symbol,,,,,,,,,,,,
A,0.001866,0.003248,0.142857,0.142857,0.142857,0.142857,0.142857,0.126767,0.016090,S1,0.281443,0.450420
B,0.017239,0.025121,0.142857,0.142857,0.142857,0.142857,0.142857,0.154675,-0.011818,S1,0.281443,0.549580
C,0.007801,0.012649,0.142857,0.142857,0.142857,0.142857,0.142857,0.155355,-0.012498,S2,0.586778,0.264760
D,0.004658,-0.004792,0.142857,0.142857,0.142857,0.142857,0.142857,0.137544,0.005313,S2,0.586778,0.234406
E,0.002034,0.002740,0.142857,0.142857,0.142857,0.142857,0.142857,0.142501,0.000356,S2,0.586778,0.242853
F,0.001931,0.026376,0.142857,0.142857,0.142857,0.142857,0.142857,0.151378,-0.008520,S2,0.586778,0.257981
G,-0.000929,0.005084,0.142857,0.142857,0.142857,0.142857,0.142857,0.131780,0.011077,S3,0.131780,1.000000
